# OpenAI Function Calling In LangChain 🤖

## Introduction
This notebook demonstrates how to `use the LangChain framework to integrate OpenAI function calling capabilities`. We'll cover the setup, the syntax of Pydantic data classes, and how to convert these data classes into OpenAI functions. Additionally, we'll explore how to bind and invoke these functions within LangChain models.

### Setup and Imports
First, we need to import the necessary libraries and set up the environment.

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
%pip install rich

In [ ]:
from rich import print
from rich.pretty import Pretty

### Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

- Creation of a class 

In [5]:
# In Python you can create a claas like that: 
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [6]:
# Example usage
foo = User(name="Joe",age=32, email="joe@gmail.com")
print(foo.name)
foo.name

In [8]:
# Example usage
foo = User(name="Joe",age="bar", email="joe@gmail.com")
foo.age

With Pydantic, you can define the class as follows:

In [10]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [11]:
# Example usage
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")
foo_p.name

In [ ]:
# Note: The following line should raise an error because 'age' should be an integer
#foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
#print(foo_p)  # Output: bar (incorrect)

In [13]:
class Class(BaseModel):
    students: List[pUser]

In [14]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [15]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


We can convert Pydantic data classes into OpenAI function definitions.

In [16]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [17]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [18]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\Michela\AppData\Local\Temp\ipykernel_1988\1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [19]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [20]:
print(Pretty(weather_function))


{
    'name': 'WeatherSearch',
    'description': 'Call this with an airport code to get the weather at that airport',
    'parameters': {
        'properties': {'airport_code': {'description': 'airport code to get weather for', 'type': 'string'}},
        'required': ['airport_code'],
        'type': 'object'
    }
}

In [21]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [ ]:
#convert_pydantic_to_openai_function(WeatherSearch1)

In [22]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [23]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [24]:
response = convert_pydantic_to_openai_function(WeatherSearch2)
print(Pretty(response))

{
    'name': 'WeatherSearch2',
    'description': 'Call this with an airport code to get the weather at that airport',
    'parameters': {
        'properties': {'airport_code': {'type': 'string'}},
        'required': ['airport_code'],
        'type': 'object'
    }
}

In [25]:
from langchain.chat_models import ChatOpenAI

In [26]:
model = ChatOpenAI()

C:\Users\Michela\AppData\Local\Temp\ipykernel_1988\1603725051.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


**Note** weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [27]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9148cc5d-0087-4bd1-9bd8-258b92f10e30-0')

In [28]:
response = model.invoke("what is the weather in SF today?", functions=[weather_function])
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 70,
            'total_tokens': 88,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-d47401c0-d3df-40db-8dc3-70f93e939d15-0'
)

In [29]:
model_with_function = model.bind(functions=[weather_function])

In [30]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 69, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f59132e8-a78a-4362-81b6-e77b09f00b42-0')

In [31]:
response = model_with_function.invoke("what is the weather in sf?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 69,
            'total_tokens': 87,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-698f88f9-75aa-4fdd-bbc9-666e7cdb77cc-0'
)

## Forcing the Model to Use a Function

We can force the model to use a specific function when invoking it.

**Note** model_with_function = model.bind(functions=[weather_function])

In [32]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [33]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 79, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a800c374-2324-4a06-9364-2798e1c3b5db-0')

In [34]:
response = model_with_forced_function.invoke("what is the weather in sf?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 8,
            'prompt_tokens': 79,
            'total_tokens': 87,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-cc94ce3d-0e5c-4f99-9a20-5cdae01e5231-0'
)

**Explanation output**  This output is a structured representation of an AIMessage object.This output indicates that the AI model invoked a function named WeatherSearch with an argument of {"airport_code":"SFO"} and provides detailed metadata about the token usage and model.  
Interesting is to have a look at the structure of the AIMessage:   
Class: AIMessage  
Attributes:  
  - content: An empty string, indicating no direct textual content in the message.    
  - additional_kwargs: A dictionary containing additional keyword arguments:  
      - function_call: A dictionary specifying:
          - arguments: A JSON string with the argument {"airport_code":"SFO"}, indicating the airport code for San Francisco.
      - name: The function name WeatherSearch.    

response_metadata: A dictionary with metadata about the response:  
  - token_usage: A dictionary detailing token usage:
completion_tokens: 8
prompt_tokens: 79
total_tokens: 87
completion_tokens_details: A dictionary with various token counts, all set to 0.
prompt_tokens_details: A dictionary with audio_tokens and cached_tokens, both set to 0.
  - model_name: The model used is gpt-3.5-turbo.
  - system_fingerprint: None.
  - finish_reason: The reason for finishing is stop.
  - logprobs: None.  
  
id: A unique identifier for the run: run-cc94ce3d-0e5c-4f99-9a20-5cdae01e5231-0.

In [35]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 74, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-37a38b34-6cf8-4b98-bac9-ffb75fed5d9e-0')

In [36]:
response = model_with_forced_function.invoke("hi!")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 8,
            'prompt_tokens': 74,
            'total_tokens': 82,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-a9f0516e-d149-4573-996a-4a4b70ecca62-0'
)

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [37]:
from langchain.prompts import ChatPromptTemplate

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [39]:
chain = prompt | model_with_function

In [40]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 75, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c53dfa35-5250-4349-b837-bed21d77efa5-0')

In [41]:
response = chain.invoke({"input": "what is the weather in sf?"})
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 75,
            'total_tokens': 93,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-8e4c77b2-02e7-4b07-bcd5-c18134787baf-0'
)

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [42]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

Let's convert Pydantic data classes to OpenAI function definitions and stores them in a list named `functions`. Each function can be used for specific tasks such as weather search and artist search in the OpenAI model.

In [43]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch), # Converts WeatherSearch Pydantic class to OpenAI function
    convert_pydantic_to_openai_function(ArtistSearch),  # Converts ArtistSearch Pydantic class to OpenAI function
]

Let's binds a list of predefined functions to an OpenAI model. The 'functions' list contains converted Pydantic data classes that define specific tasks (e.g., WeatherSearch, ArtistSearch). Binding these functions to the model enables the model to invoke them during its execution for specific queries.

In [44]:
model_with_functions = model.bind(functions=functions)

Next, let's  snippet invoke the `model_with_functions` with the query "what is the weather in sf?". The model is expected to use the bound functions (e.g., WeatherSearch) to process this query. The query will trigger the model to call the appropriate function to get the weather information for San Francisco (SFO).

In [45]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 116, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f43d22a7-e11b-4cfa-aa92-5f821517a764-0')

In [46]:
response = model_with_functions.invoke("what is the weather in sf?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 116,
            'total_tokens': 134,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-547504bf-8d94-4c18-ac35-ef6852458399-0'
)

In [47]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 118, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1879bf7e-6787-478a-b9f4-258dd491e72d-0')

In [48]:
response = model_with_functions.invoke("what are three songs by taylor swift?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 22,
            'prompt_tokens': 118,
            'total_tokens': 140,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-570e8342-f07c-449b-b87d-f5ceb17bfd1f-0'
)

In [49]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 111, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7ceefa2d-b257-4426-a229-6dcad1886997-0')

In [50]:
response = model_with_functions.invoke("hi!")
print(Pretty(response))

AIMessage(
    content='Hello! How can I assist you today?',
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 11,
            'prompt_tokens': 111,
            'total_tokens': 122,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-a781a28a-0a94-4c21-b817-c8a992961894-0'
)

## Conclusion
In this notebook, we demonstrated how to `use LangChain to integrate OpenAI function calling capabilities`. We covered the setup, Pydantic data classes, converting these classes into OpenAI functions, and invoking these functions within LangChain models. These examples showcase the flexibility and power of LangChain in building advanced language models and workflows.